In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class TestModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.mlp = nn.Linear(2,1)
        self.reset()
    def forward(self,X):
        return self.mlp(X)
    def reset(self):
        # 对weight全1初始化、bias全0初始化，便于之后的研究
        self.mlp.weight.data.fill_(1)
        self.mlp.bias.data.fill_(0)


model.eval()
- 关闭Dropout层
- BatchNorm层停止更新均值和方差
- 仍然会计算梯度，只是不进行反向传播

model.train()
- 开启Dropout层
- BatchNorm层保持更新均值和方差

with torch.no_grad()
- 不开启梯度的反向传播，节省计算资源

In [2]:
model = TestModel()
optimizer = optim.Adam(model.parameters(),lr=0.5)

X = torch.tensor([[0,2]]).float()
Y = torch.tensor([[1]]).float()

Z = model(X)
print("由于weight全1bias全0，输出的Z为：{}\n".format(Z.detach()))

loss = F.mse_loss(Z,Y)
print("前向传播时，梯度为：{}\n".format(model.mlp.weight.grad))

loss.backward()
print("反向传播时，梯度为：{}\n".format(model.mlp.weight.grad))

# 测试no_grad()
print("在with no_grad()中使用loss.backward()会导致报错\n")

# # 取消注释以查看报错信息
# with torch.no_grad():
#     Z = model(X)
#     loss = F.mse_loss(Z,Y)
#     loss.backward()

optimizer.step()
model.zero_grad()
print("梯度已清空：{}".format(model.mlp.weight.grad))
print("参数更新为：{}".format(model.mlp.weight.data))
print("model.zero_grad()和optimizer.step()都会清空参数的梯度，区别在于一个不训练参数，一个训练\n")

# 把模型参数恢复为初始状态
model.reset()

# 测试eval
model.eval()
Z = model(X)
loss = F.mse_loss(Z,Y)
loss.backward()
print("当前梯度：{}".format(model.mlp.weight.grad))
print("可见，eval()模式下梯度仍会更新")

Z = model(X)
loss = F.mse_loss(Z,Y)
loss.backward()
print("如果loss更新后没有optimizer.step()，那么新梯度会叠加在旧梯度上：{}\n".format(model.mlp.weight.grad))

print("更新前：{}".format(model.mlp.weight.data))
optimizer.step()
print("更新后：{}".format(model.mlp.weight.data))
print("eval()模式下，仍然可以手动进行optimizer.step()\n")




由于weight全1bias全0，输出的Z为：tensor([[2.]])

前向传播时，梯度为：None

反向传播时，梯度为：tensor([[0., 4.]])

在with no_grad()中使用loss.backward()会导致报错

梯度已清空：None
参数更新为：tensor([[1.0000, 0.5000]])
model.zero_grad()和optimizer.step()都会清空参数的梯度，区别在于一个不训练参数，一个训练

当前梯度：tensor([[0., 4.]])
可见，eval()模式下梯度仍会更新
如果loss更新后没有optimizer.step()，那么新梯度会叠加在旧梯度上：tensor([[0., 8.]])

更新前：tensor([[1., 1.]])
更新后：tensor([[1.0000, 0.5174]])
eval()模式下，仍然可以手动进行optimizer.step()



装饰器

model.no_grad()与optimizer.no_grad()在单台机器单张显卡时没有区别
这里无法演示，跳过